In [ ]:
import netCDF4
import numpy as np
import numpy.ma as ma
import os
import matplotlib.pyplot as plt

Specify a KAZRcfr spectra data file, "myfile"

In [ ]:
myfile = '/Users/karenjohnson/arm_data/mos/moskazrcfrspcmdcopolM1.a1/moskazrcfrspcmdcopolM1.a1.20200929.080005.nc'

Let's open the file and look at the dimensions 

In [ ]:
cdf = netCDF4.Dataset(myfile)
to = cdf.variables['time'][:]
rg = cdf.variables['range'][:]
num_times = len(to)
num_gates = len(rg)
print('Number of time profiles = ', num_times)
print('Minimum and maximum ranges are ', rg[0], rg[-1])

Set time profile and range min/max values to plot.
Optionally, you can also set the contour min, max for the spectrogram 
(may be best to use defaults for the 'first look')

In [ ]:
time_index_to_plot = 20
range_min = 500.
range_max = 10000.
contour_limits = None
#contour_min = -90.
#contour_max = -60
#contour_limits = [contour_min, contour_max]

Read in spectra and other info 

In [ ]:
justfile = os.path.basename(myfile)
dotchunks = justfile.split('.')
spcpos = dotchunks[0].find('spc')
if spcpos == -1:
    print('This code only works for kazrcfrspc files. Sorry!')
    raise TypeError("Wrong type of data file")

Extract info from the file name

In [ ]:
thesite = justfile[0:3]
facil = dotchunks[0][-2:]
facilpos = dotchunks[0].find(facil)
yyyymmdd = dotchunks[2]
hhmmss = dotchunks[3]
mode = dotchunks[0][spcpos+3:spcpos+5]
radar = dotchunks[0][3:spcpos]
pol = dotchunks[0][spcpos+5:facilpos]

Read more data from the file itself

In [ ]:
loc = cdf.variables['spectrum_index'][:,:]
if pol == 'copol':
    spectra = cdf.variables['radar_power_spectrum_of_copolar_h'][:,:]
else:
    spectra = cdf.variables['radar_power_spectrum_of_crosspolar_v'][:,:]
 
# Determine yquist velocity bins
nyqtime = cdf.variables['nyquist_velocity'][:]
nfft = cdf.fft_len
nyq = nyqtime[0]
vdelta = nyq/(nfft/2.)
vbins = np.arange(-nyq, nyq, nyq/(nfft/2))


*** Plotting Spectrogram ***
1. Find range gate indices needed for range_min,range_max

In [ ]:
rb_j = np.argmin(np.abs(rg - range_min))
re_j = np.argmin(np.abs(rg - range_max))

2.  Selecct spectra needed for desired spectorgram time 

In [ ]:
spcy = np.empty((re_j - rb_j + 1, nfft,))
spcy[:] = np.NAN
for j in range(rb_j, re_j+1):
    if(loc[time_index_to_plot, j] is not ma.masked):
        spcy[j,:] = spectra[loc[time_index_to_plot, j], :]

3.  Check for and fix possible bad spectral value(s) -- This can cause poor countour selections if not corrected

In [ ]:
bad_index = np.where(spcy == -32767)
if len(spcy[bad_index]) > 0:
    spcy[bad_index] = np.nan
smin = np.floor(np.nanmin(spcy))
smax = np.ceil(np.nanmax(spcy))
print('Min and Max of spectra value: ', smin, smax)
if contour_limits:
    smin, smax = contour_limits
print('Contour limits will be: ', smin, smax)

Construct plot titles and plot spectrogam

In [ ]:
str_time = str(round(to[time_index_to_plot]/30., 2))    
titlefig = thesite + '.' + facil + ' ' + radar + mode + pol
titlestr = yyyymmdd + '.' + hhmmss + ' plus ' + str_time + 'min (index ' + str(time_index_to_plot)+ ')' 

plt.figure(figsize=[6,8])
plt.pcolor(vbins, rg[rb_j:re_j+1], spcy, vmin=smin, vmax=smax)
plt.title(titlestr)
plt.suptitle(titlefig)
plt.xlabel('velocity (m/s)')
plt.ylabel('range (m)')
plt.ylim([range_min, range_max])

plt.colorbar(orientation='vertical',shrink=0.8)
plt.show()